# TI3145TU Midterm Assignment 
## Football Players Wages

We hope you enjoy this assignment, good luck!

Student names: XXX

Student numbers: XXX

### Imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor

import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

### Data Loading and Preprocessing

Data is read from the csv file 'football_wages.csv'.

To convert the nationaility_name column from strings to numeric data, each nationality is assigned an integer, based on its ranking of average log_wages.

Since the dataset is significantly smaller than the "football_autograder.csv", all the data is kept from "football_wages.csv" for training and validation. Overfitting is minimized by means of cross validation. Then the error estimation is done by averaging the predictions of the models over the entire Y set, which they have been tunned for different data points. 

In [15]:
data = pd.read_csv('football_wages.csv')
data_autograder = pd.read_csv('football_autograder.csv')



sort_nationalities = True

if sort_nationalities == True:
    # Group by nationality_name and calculate the mean for all features
    mean_by_nationality = data.groupby('nationality_name').mean().reset_index()

    # Divide each feature's mean by the mean of log_wages
    for column in mean_by_nationality.columns:
        if column != 'nationality_name' and column != 'log_wages':
            mean_by_nationality[column] = mean_by_nationality[column] / mean_by_nationality['log_wages']

    print("\nMean of features by nationality (divided by log_wages): ", mean_by_nationality)

    # Sort the data based on the new calculated values
    mean_by_nationality_sorted = mean_by_nationality.sort_values(by='log_wages', ascending=False)

    print("\nSorted nationalities: ", mean_by_nationality_sorted)

    # Merge the sorted means back with the original data
    sorted_data = pd.merge(data, mean_by_nationality_sorted, on='nationality_name', suffixes=('', '_mean'))
    sorted_data = sorted_data.sort_values(by='log_wages_mean', ascending=False).drop(columns='log_wages_mean')

    # Convert the nationality strings to integers, with 0 being the country with lowest average log_wages
    sorted_data['nationality_name'], uniques = pd.factorize(data['nationality_name'])

    X = sorted_data.iloc[:, :-1]

else:
    data.drop('nationality_name',axis=1,inplace=True)
    X = data.iloc[:, :-1]


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Y = sorted_data.iloc[:, -1]
X_train, X_val, y_train, y_val = train_test_split(X_scaled, Y, test_size=0.3, random_state=100)




Mean of wages by nationality:             nationality_name  log_wages
0                b'Albania'   3.454647
1                b'Algeria'   4.087022
2                 b'Angola'   3.782313
3    b'Antigua and Barbuda'   3.301030
4              b'Argentina'   3.629543
..                      ...        ...
126            b'Venezuela'   2.943126
127              b'Vietnam'   3.301030
128                b'Wales'   3.590578
129               b'Zambia'   2.698970
130             b'Zimbabwe'   3.570825

[131 rows x 2 columns]

Sorted nationalities:              nationality_name  log_wages
115              b'Tanzania'   4.633468
5                 b'Armenia'   4.456907
31                   b'Cuba'   4.361728
13                b'Bermuda'   4.301030
61                 b'Israel'   4.255273
..                       ...        ...
58                  b'India'   2.707986
14                b'Bolivia'   2.707824
129                b'Zambia'   2.698970
122  b'United Arab Emirates'   2.698970
36     b'Dom

In [15]:
sorted_data.head()

,age,height_cm,weight_kg,nationality_name,overall,potential,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,...,movement_reactions,movement_balance,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,log_wages
2999,28.0,183.0,80.0,0,74.0,74.0,69.0,69.0,75.0,76.0,...,72.0,69.0,28.0,23.0,12.0,14.0,9.0,7.0,9.0,4.633468
860,25.0,186.0,82.0,1,70.0,75.0,61.0,69.0,69.0,64.0,...,67.0,74.0,14.0,20.0,5.0,9.0,7.0,9.0,5.0,4.000000
4564,32.0,177.0,75.0,0,83.0,83.0,77.0,80.0,68.0,83.0,...,82.0,83.0,53.0,55.0,5.0,11.0,14.0,5.0,7.0,4.913814
2511,28.0,172.0,76.0,2,71.0,71.0,64.0,63.0,50.0,65.0,...,65.0,85.0,38.0,17.0,14.0,13.0,10.0,11.0,11.0,4.361728
3330,31.0,170.0,69.0,3,70.0,70.0,60.0,72.0,68.0,64.0,...,71.0,78.0,43.0,35.0,10.0,6.0,13.0,11.0,7.0,4.301030


### Define ftraining functions

In [16]:
def train_and_test_knn(k, x_train, y_train, x_val,y_val):
    # Initialize the KNN regressor
    model = KNeighborsRegressor(n_neighbors=k)

    # Train the KNN regressor
    model.fit(x_train, y_train)

    # Predict on the validation set
    y_pred = model.predict(x_val)

    # Calculate error
    error = np.mean(np.abs(y_val - y_pred))

    return model,error

def tune_and_store_knn(x_train,y_train,x_val,y_val):
    min_error = np.inf
    for k in range(1,40):
        model,error = train_and_test_knn(k,x_train,y_train,x_val,y_val)
        #print('K: ',k,' Error: ',error)

        if error < min_error:
            min_error = error
            best_k = k
            best_model = model
    print("Best K:", best_k)
    return best_k,best_model,min_error

def train_and_test_SGD(parameters,x_train,y_train,x_val,y_val): # To be implemented by Edlyn
    return
def tune_and_store_SGD(x_train,y_train,x_val,y_val): # To be implemented by Edlyn
    return

### Train and store tunned KNNs

In [17]:
#########################################
#--------KNN Cross validation Training 
#########################################
number_of_models = 15
best_KNN_model_list =[]
best_k_list = []
val_error_list = []

# Perform cross validation
for i in range(number_of_models):
    print("-----\nModel: ", i)
    x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size=0.2,random_state=i)
    best_k,best_model,min_error = tune_and_store_knn(x_train,y_train,x_val,y_val)
    best_KNN_model_list.append(best_model)
    val_error_list.append(min_error)
    best_k_list.append(best_k)

print('Best K list: ',best_k_list)
norm_val_error = np.mean(np.array(val_error_list))
print('Normalized MAE on validation sets: ',norm_val_error)


-----
Model:  0
Best K: 7
-----
Model:  1
Best K: 15
-----
Model:  2
Best K: 10
-----
Model:  3
Best K: 17
-----
Model:  4
Best K: 10
-----
Model:  5
Best K: 16
-----
Model:  6
Best K: 9
-----
Model:  7
Best K: 12
-----
Model:  8
Best K: 8
-----
Model:  9
Best K: 18
-----
Model:  10
Best K: 10
-----
Model:  11
Best K: 11
-----
Model:  12
Best K: 11
-----
Model:  13
Best K: 17
-----
Model:  14
Best K: 11
Best K list:  [7, 15, 10, 17, 10, 16, 9, 12, 8, 18, 10, 11, 11, 17, 11]
Normalized MAE on validation sets:  0.2888334506868395


In [7]:
ks = [1, 2, 5, 10, 20, 30, 50, 100]

err_test = []

for s in ks:
    knc = KNeighborsRegressor(n_neighbors=s)

    knc.fit(X_train, y_train)

    test_predictions = knc.predict(X_test)
    train_predictions = knc.predict(X_train)
    
    err_test.append(mean_absolute_error(y_test, test_predictions))

# END ANSWER

best_k_index = np.argmin(err_test)
best_k = ks[best_k_index]
print("The best k on the test set is %d" % best_k)
print("The lowest error is ", err_test[best_k_index])
print(10 ** err_test[best_k_index])


In [22]:
# Instantiate the SGDRegressor
sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)

# Define the parameter grid for alpha (regularization strength) and learning rate
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Tuning regularization strength
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],  # Different learning rate strategies
    'eta0': [0.001, 0.01, 0.1, 1]  # Initial learning rate for strategies that require it
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(sgd_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_scaled, y_train)

# Get the best model from grid search
best_sgd = grid_search.best_estimator_

# Predict on the test set with the best model
y_pred = best_sgd.predict(X_test_scaled)

# Evaluate the model using MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Best MAE: {mae}")
print(f"Best Parameters: {grid_search.best_params_}")

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stocha

Best MAE: 0.27692806009280757
Best Parameters: {'alpha': 0.0001, 'eta0': 0.1, 'learning_rate': 'adaptive'}


In [ ]:
# TODO Replace this with your own estimate of the MAE of your best model
estimate_MAE_on_new_data = np.array([1.0])

# TODO Replace this with the predictions of your best model
# via e.g. prediction = model.predict(data_autograder)
# your predictions here should again be the $log_{10}(wage)$ of the football player, just as in the provided data. 
predictions_autograder_data = np.array([-1] * 14178)

# Upload this file to the Vocareum autograder:
result = np.append(estimate_MAE_on_new_data, predictions_autograder_data)
pd.DataFrame(result).to_csv("autograder_submission.txt", index=False, header=False)